Segmenting and Clustering Neighborhoods in Toronto-Part1

In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
! pip install --user bs4
! pip install --user parse
! pip install --user lxml
! pip install --user geocoder
import parse
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from bs4 import BeautifulSoup

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [4]:
soup = BeautifulSoup(html, 'html.parser')


In [5]:
tables = soup.find('table')
for a in soup.findAll('a'):
    a.replaceWithChildren()


In [6]:
rows = soup.find_all('tr')
for row in rows:
    row_td = tables.find_all('td')
len(row_td)



864

In [7]:

i=0
fields,details,details1=[],[],[]
while(i<=(len(row_td)-3)):
    fields.append(str(row_td[i]))
    details.append(str(row_td[i+1]))
    details1.append(str(row_td[i+2]))
    i=i+3
df=pd.DataFrame({'PostalCode':fields,'Borough':details,'Neighbourhood':details1})
print(df)

       PostalCode                    Borough                     Neighbourhood
0    <td>M1A</td>      <td>Not assigned</td>           <td>Not assigned\n</td>
1    <td>M2A</td>      <td>Not assigned</td>           <td>Not assigned\n</td>
2    <td>M3A</td>        <td>North York</td>              <td>Parkwoods\n</td>
3    <td>M4A</td>        <td>North York</td>       <td>Victoria Village\n</td>
4    <td>M5A</td>  <td>Downtown Toronto</td>           <td>Harbourfront\n</td>
..            ...                        ...                               ...
283  <td>M8Z</td>         <td>Etobicoke</td>              <td>Mimico NW\n</td>
284  <td>M8Z</td>         <td>Etobicoke</td>     <td>The Queensway West\n</td>
285  <td>M8Z</td>         <td>Etobicoke</td>  <td>Royal York South West\n</td>
286  <td>M8Z</td>         <td>Etobicoke</td>         <td>South of Bloor\n</td>
287  <td>M9Z</td>      <td>Not assigned</td>           <td>Not assigned\n</td>

[288 rows x 3 columns]


In [8]:
for col in df.columns:
    df[col] = df[col].str.replace("<td>", "")
    df[col] = df[col].str.replace("</td>", "")
    df[col] = df[col].str.replace("\n", "")
    

In [9]:
df

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [10]:
df=df[df['Borough']!='Not assigned']

In [11]:
df

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [12]:
index=df['Neighbourhood']=='Not assigned'
df['Neighbourhood'][index]=df['Borough'][index]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
df_new = df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [14]:
df_new

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


Segmenting and Clustering Neighborhoods in Toronto-Part2

In [15]:
coords=pd.read_csv("Geospatial_Coordinates.csv")

In [16]:
coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [17]:
df_new['Latitude']=coords['Latitude']
df_new['Longitude']=coords['Longitude']

In [18]:
df_new

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


<h1>Segmenting and Clustering Neighborhoods in Toronto-Part3

In [22]:
df1 = df_new[df_new['Borough'].str.contains("Toronto")] 

In [33]:
df1.shape

(38, 5)

In [29]:
b = pd.DataFrame(df1.Neighbourhood.str.split(',').tolist(), index=[df1.Latitude,df1.Longitude]).stack()


In [30]:
b

Latitude   Longitude    
43.676357  -79.293031  0                                          The Beaches
43.679557  -79.352188  0                                    The Danforth West
                       1                                            Riverdale
43.668999  -79.315572  0                                     The Beaches West
                       1                                         India Bazaar
                                                  ...                        
43.648960  -79.456325  0                                             Parkdale
                       1                                         Roncesvalles
43.651571  -79.484450  0                                            Runnymede
                       1                                              Swansea
43.662744  -79.321558  0    Business Reply Mail Processing Centre 969 Eastern
Length: 74, dtype: object

In [31]:
b = b.reset_index()[[0, 'Latitude','Longitude']]

In [34]:
b.columns=['Neighbourhood','Latitude','Longitude']

In [35]:
b

,Neighbourhood,Latitude,Longitude
0,The Beaches,43.676357,-79.293031
1,The Danforth West,43.679557,-79.352188
2,Riverdale,43.679557,-79.352188
3,The Beaches West,43.668999,-79.315572
4,India Bazaar,43.668999,-79.315572
...,...,...,...
69,Parkdale,43.648960,-79.456325
70,Roncesvalles,43.648960,-79.456325
71,Runnymede,43.651571,-79.484450
72,Swansea,43.651571,-79.484450


In [36]:
CLIENT_ID = 'HVAL5O1T5EUXSJOBOILP00IIDDQV2KLKMF402HIVWP1TRMGU' # your Foursquare ID
CLIENT_SECRET = 'FNT50QAGMQ4NRYWQ5LD2Z3MJY25HIOPOYHNS4JBKTX2N0IH1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [42]:
import requests
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [43]:
toronto_venues = getNearbyVenues(names=b['Neighbourhood'],
                                   latitudes=b['Latitude'],
                                   longitudes=b['Longitude']
                                  )



The Beaches
The Danforth West
 Riverdale
The Beaches West
 India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
 Summerhill East
Deer Park
 Forest Hill SE
 Rathnelly
 South Hill
 Summerhill West
Rosedale
Cabbagetown
 St. James Town
Church and Wellesley
Harbourfront
 Regent Park
Ryerson
 Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide
 King
 Richmond
Harbourfront East
 Toronto Islands
 Union Station
Design Exchange
 Toronto Dominion Centre
Commerce Court
 Victoria Hotel
Roselawn
Forest Hill North
 Forest Hill West
The Annex
 North Midtown
 Yorkville
Harbord
 University of Toronto
Chinatown
 Grange Park
 Kensington Market
CN Tower
 Bathurst Quay
 Island airport
 Harbourfront West
 King and Spadina
 Railway Lands
 South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place
 Underground city
Christie
Dovercourt Village
 Dufferin
Little Portugal
 Trinity
Brockton
 Exhibition Place
 Parkdale Village
High Park
 The 

In [45]:
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Danforth West,43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
...,...,...,...,...,...,...,...
3313,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,The Ten Spot,43.664815,-79.324213,Spa
3314,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
3315,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
3316,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop


In [48]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Danforth West,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3313,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3314,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3315,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3316,Business Reply Mail Processing Centre 969 Eastern,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Bathurst Quay,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Dufferin,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Exhibition Place,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.043478
3,Forest Hill SE,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.066667,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.066667,0.0,0.0,0.0,0.000000
4,Forest Hill West,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.25000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Studio District,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.051282,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.025641
70,The Annex,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.045455,0.0,...,0.0,0.00000,0.0,0.045455,0.0,0.000000,0.0,0.0,0.0,0.000000
71,The Beaches,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.25000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
72,The Beaches West,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [50]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [51]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    print(ind)
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

0
1
2
3
4
5
6
7
8
9


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Quay,Airport Service,Airport Lounge,Airport Terminal,Boat or Ferry,Harbor / Marina,Sculpture Garden,Coffee Shop,Boutique,Bar,Airport Gate
1,Dufferin,Pharmacy,Bakery,Supermarket,Bank,Pet Store,Bar,Gym / Fitness Center,Park,Brewery,Café
2,Exhibition Place,Coffee Shop,Breakfast Spot,Café,Climbing Gym,Stadium,Burrito Place,Sandwich Place,Caribbean Restaurant,Restaurant,Yoga Studio
3,Forest Hill SE,Pub,Coffee Shop,Pizza Place,Light Rail Station,Sports Bar,Supermarket,Sushi Restaurant,Bagel Shop,Restaurant,Fried Chicken Joint
4,Forest Hill West,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [52]:
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Bathurst Quay,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
1,Dufferin,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Exhibition Place,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.043478
3,Forest Hill SE,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.066667,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.066667,0.0,0.0,0.0,0.000000
4,Forest Hill West,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.25000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Studio District,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.051282,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.025641
70,The Annex,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.045455,0.0,...,0.0,0.00000,0.0,0.045455,0.0,0.000000,0.0,0.0,0.0,0.000000
71,The Beaches,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.25000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000
72,The Beaches West,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.0,...,0.0,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000


In [56]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 1, 1, 4, 1, 1, 3, 1, 3], dtype=int32)

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = b

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Trail,Neighborhood,Event Space,Ethiopian Restaurant,Electronics Store,Falafel Restaurant,Eastern European Restaurant,Dessert Shop
1,The Danforth West,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Sports Bar,Juice Bar,Spa,Liquor Store
2,Riverdale,43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Sports Bar,Juice Bar,Spa,Liquor Store
3,The Beaches West,43.668999,-79.315572,1,Park,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Movie Theater,Ice Cream Shop,Fish & Chips Shop,Pub,Intersection,Italian Restaurant
4,India Bazaar,43.668999,-79.315572,1,Park,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Movie Theater,Ice Cream Shop,Fish & Chips Shop,Pub,Intersection,Italian Restaurant


In [64]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[43.676357,-79.293031], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters